In [2]:
!pip install pandas
!pip install geopy
!pip install requests
!pip install matplotlib
!pip install sklearn
!pip install folium
!pip install lxml 
!pip install lxml
!pip install bs4
!pip install openpyxl 

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import numpy as np

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests
from bs4 import BeautifulSoup 

## Get raw data

### Raw data-Travel City 1

In [70]:
# scrape Wikipedia page of Toronto (postal code M)
def get_toronto_raw():
    url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
    page_tables  = pd.read_html(url)
    df_toronto_raw = page_tables[0]
    return df_toronto_raw

df_toronto_raw = get_toronto_raw()

In [5]:
df_toronto_raw[:3]

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...


Modeling the raw data

In [71]:
def get_list_canada_zipcode(df_toronto_raw):
    
    list_canada_zipcode = []
    for index in range(len(df_toronto_raw.columns)):
        serie = df_toronto_raw.iloc[:, index]
        list_col = serie.tolist()
        list_canada_zipcode += list_col
    return list_canada_zipcode

list_canada_zipcode = get_list_canada_zipcode(df_toronto_raw)

In [72]:
list_canada_zipcode[:10]

['M1ANot assigned',
 'M1BScarborough(Malvern / Rouge)',
 'M1CScarborough(Rouge Hill / Port Union / Highland Creek)',
 'M1EScarborough(Guildwood / Morningside / West Hill)',
 'M1GScarborough(Woburn)',
 'M1HScarborough(Cedarbrae)',
 'M1JScarborough(Scarborough Village)',
 'M1KScarborough(Kennedy Park / Ionview / East Birchmount Park)',
 'M1LScarborough(Golden Mile / Clairlea / Oakridge)',
 'M1MScarborough(Cliffside / Cliffcrest / Scarborough Village West)']

In [73]:
def get_list_dict_canada_zipcode(list_canada_zipcode):
    
    list_dict = []
    for item in list_canada_zipcode:
        d = {}
        list_data = item.split('(')
        if 'Not assigned' in list_data[0]:
            postal_code = item.replace('Not assigned', '')
        d['postal_code'] = list_data[0][:3]
        d['borough'] = list_data[0][3:]
        try: 
            list_neighborhood = list_data[1].replace(')', '').replace(' / ', ', ')
            d['neighborhood'] = list_neighborhood
        except:
            d['neighborhood'] = ''
        list_dict.append(d)
    return list_dict

list_dict_canada_zipcode = get_list_dict_canada_zipcode(list_canada_zipcode)

In [74]:
list_dict_canada_zipcode[:5]

[{'postal_code': 'M1A', 'borough': 'Not assigned', 'neighborhood': ''},
 {'postal_code': 'M1B',
  'borough': 'Scarborough',
  'neighborhood': 'Malvern, Rouge'},
 {'postal_code': 'M1C',
  'borough': 'Scarborough',
  'neighborhood': 'Rouge Hill, Port Union, Highland Creek'},
 {'postal_code': 'M1E',
  'borough': 'Scarborough',
  'neighborhood': 'Guildwood, Morningside, West Hill'},
 {'postal_code': 'M1G', 'borough': 'Scarborough', 'neighborhood': 'Woburn'}]

In [75]:
def get_df_canada_zipcode(list_dict):
    
    df_canada_raw = pd.DataFrame(list_dict)
    maskna = df_canada_raw['borough'].isin(['Not assigned'])
    df_canada = df_canada_raw.loc[~maskna, :]
    return df_canada


,postal_code,borough,neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [85]:

df_canada = get_df_canada_zipcode(list_dict_canada_zipcode)
df_canada[:5]

,postal_code,borough,neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


Filtering zipcode of Toronto city only

In [76]:
def get_df_toronto_zipcode(df_canada):

    mask_toronto = df_canada['borough'].str.contains('Toronto')
    df_toronto = df_canada.loc[mask_toronto, :].reset_index(drop=True)
    return df_toronto

df_toronto_raw = get_df_toronto_zipcode(df_canada)

print(f'Postal Code count: {df_toronto_raw.shape[0]}')
df_toronto_raw.head()

Postal Code count: 39


,postal_code,borough,neighborhood
0,M4E,East Toronto,The Beaches
1,M4J,East YorkEast Toronto,The Danforth East
2,M4K,East Toronto,"The Danforth West, Riverdale"
3,M4L,East Toronto,"India Bazaar, The Beaches West"
4,M4M,East Toronto,Studio District


### Raw data-Travel City 2

In [77]:
def get_html_page(url):
    html_page = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text
    return html_page

def parser_html(html_page):
    bs = BeautifulSoup(html_page, 'html.parser')
    return bs


In [78]:
def get_html_paris_parsed():

    url = 'https://worldpostalcode.com/france/ile-de-france/paris'
    html_page = get_html_page(url)
    bs = parser_html(html_page)
    return bs

html_paris = get_html_paris_parsed()

In [79]:
def get_divs_borough(bs):
    divs_borough = bs.findAll("div", {"class": "code"})
    return divs_borough

divs_borough = get_divs_borough(html_paris)


looking the html raw data

In [88]:
divs_borough[0]

<div class="code"><span>75000</span> <span>75001</span> <span>75002</span> <span>75003</span> <span>75004</span> <span>75005</span> <span>75006</span> <span>75007</span> <span>75008</span> <span>75009</span> <span>75010</span> <span>75011</span> <span>75012</span> <span>75013</span> <span>75014</span> <span>75015</span> <span>75016</span> <span>75017</span> <span>75018</span> <span>75019</span> <span>75020</span></div>

In [80]:
def get_postal_code_neighborhood(div_borough):
    
    spans = div_borough.findAll('span')
    
    list_postal_code = []
    for index, span in enumerate(spans):
        dict_postal_code = {}
        if index == 0:
            borough = span.text
        else:
            dict_postal_code['borough'] = borough
            dict_postal_code['neighborhood'] = span.text
            list_postal_code.append(dict_postal_code)
    return list_postal_code


In [81]:
def get_list_postal_code(divs_borough):

    list_postal_code = []
    for index, div_borough in enumerate(divs_borough):
        if index > 0:
            list_postal_code_neighborhood = get_postal_code_neighborhood(div_borough)
            list_postal_code += list_postal_code_neighborhood
    return list_postal_code

In [82]:
def get_df_paris_raw(divs_borough):
    
    list_postal_code = get_list_postal_code(divs_borough)
    df = pd.DataFrame(list_postal_code)
    return df



Converting from html format to table format

In [86]:
df_paris_raw_unprocess = get_df_paris_raw(divs_borough)
df_paris_raw_unprocess.head()

,borough,neighborhood
0,75001,75021 CEDEX 01
1,75001,75023 CEDEX 01
2,75001,75024 CEDEX 01
3,75001,75026 CEDEX 01
4,75001,75028 CEDEX 01


In [83]:
def process_df_paris_raw(df_paris_raw_unprocess):
    
    df_paris_raw = df_paris_raw_unprocess.copy()
    df_paris_raw['postal_code'] = df_paris_raw['neighborhood'].str.split(' ',1, expand=True)[0]
    df_paris_raw.head()
    return df_paris_raw

df_paris_raw = process_df_paris_raw(df_paris_raw_unprocess)
print(f'Postal Code count: {df_paris_raw.shape[0]}')
df_paris_raw.head()

Postal Code count: 825


,borough,neighborhood,postal_code
0,75001,75021 CEDEX 01,75021
1,75001,75023 CEDEX 01,75023
2,75001,75024 CEDEX 01,75024
3,75001,75026 CEDEX 01,75026
4,75001,75028 CEDEX 01,75028


### Raw data-Resident City

In [45]:
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data_raw = json.load(json_data)

Data source initially in json format

In [64]:
newyork_data = newyork_data_raw['features']
newyork_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [47]:
column_names = ['borough', 'neighborhood', 'latitude', 'longitude'] 
df_newyork = pd.DataFrame(columns=column_names)
for data in newyork_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_newyork = df_newyork.append({'borough': borough,
                                    'neighborhood': neighborhood_name,
                                    'latitude': neighborhood_lat,
                                    'longitude': neighborhood_lon}, ignore_index=True)

In [48]:
qt_nbh_ny = df_newyork.shape[0]
print(f'total neighborhood: {qt_nbh_ny}')
df_newyork.head()

total neighborhood: 306


,borough,neighborhood,latitude,longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Standardized data in table format

In [89]:
df_newyork.head()

,borough,neighborhood,latitude,longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Fill coordenates from postal_code

In [27]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Set connection params to api forsquare

In [84]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20180604'
LIMIT = 30


### create function to get coordenates from address (zipcode)

From these postal codes, together with the Geopy geo-location lib, it was possible to obtain the coordinates of each postal code.

In [29]:
def get_corr(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = 0
    longitude = 0
    try:
        latitude = location.latitude
        longitude = location.longitude
    except:
        print(f'error at address: {address}')
        
    # print(latitude, longitude)
    return (latitude, longitude)

### Test function with a sample postal_code

In [30]:
address = '75021'
corr = get_corr(address)
corr

(33.739566601214094, -96.51453848227241)

### Generate coordenates from postal_codes

In [31]:
def get_df_with_coor(df_raw):
    df_with_coor =df_raw.copy()
    df_with_coor['latitude'] = ''
    df_with_coor['longitude'] = ''
    for index, address in enumerate(df_raw['postal_code']):

        corr = get_corr(address)
        df_with_coor.loc[index, 'latitude'] = corr[0]
        df_with_coor.loc[index, 'longitude'] = corr[1]
    return df_with_coor



### Fill coordenates-Travel city 1

In [32]:
def get_df_toronto(df_toronto_raw):
    # Get Geospatial_Coordinates data table
    df_geo_coor = pd.read_csv('Geospatial_Coordinates.csv')
    toronto_data = df_toronto_raw.join(df_geo_coor.set_index('Postal Code'), on='postal_code')
    toronto_data.columns= toronto_data.columns.str.lower()
    df_toronto = toronto_data
    return df_toronto
    
df_toronto = get_df_toronto(df_toronto_raw)
df_toronto.head()

,postal_code,borough,neighborhood,latitude,longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


### Fill coordenates-Travel city 2

In [31]:
def get_df_paris(df_paris_raw):
    df_paris = df_paris_raw.copy()
    df_paris['latitude'] = ''
    df_paris['longitude'] = ''
    for index, address in enumerate(df_paris_raw['postal_code']):

        corr = get_corr(address)
        df_paris.loc[index, 'latitude'] = corr[0]
        df_paris.loc[index, 'longitude'] = corr[1]
    
df_paris = get_df_paris(df_paris_raw)
print(df_paris.head())

error at address: 75634
error at address: 75729
error at address: 75815
error at address: 75817
error at address: 75872
error at address: 75893
error at address: 75983
error at address: 75984
error at address: 75985
  borough    neighborhood postal_code   latitude  longitude
0   75001  75021 CEDEX 01       75021  33.739567 -96.514538
1   75001  75023 CEDEX 01       75023    33.0572 -96.746215
2   75001  75024 CEDEX 01       75024  59.113624  25.318478
3   75001  75026 CEDEX 01       75026  32.845431 -96.771442
4   75001  75028 CEDEX 01       75028  33.035782 -97.061249


**Note:** In order not to overload the searchs methods, the captured data was saved locally to enable future analysis without having to use the API again.

In [34]:
df_paris.to_excel('df_paris.xlsx', index=False)

In [33]:
df_paris = pd.read_excel('df_paris.xlsx', engine='openpyxl')


### Fill coordenates-Resident city

For New York city, the data source already had coordinate data

In [41]:
df_newyork.head()

,borough,neighborhood,latitude,longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Extract venues data

From these coordinates together with the FourSquare geo-location API, with support for searching for commercial data, a search for commercial establishments within a radius of 500 meters (5) was applied, thus enriching the database with names of establishments and their categories.


### Create funcion to extract venues and category info

In [42]:
# function that extracts category of each venue of each Neighborhood and compound a dataframe
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        try:
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            len_categories = len(results)
            print(f'{name} - {len_categories}')

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                          'Neighborhood Latitude', 
                          'Neighborhood Longitude', 
                          'Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']
        except:
            pass
    return(nearby_venues)

### Fill Venues-Travel City 1

In [43]:
toronto_venues = getNearbyVenues(names=df_toronto['neighborhood'],
                                 latitudes=df_toronto['latitude'],
                                 longitudes=df_toronto['longitude'])

{'meta': {'code': 200, 'requestId': '605db724a17fd845dfc60a4b'}, 'response': {'headerLocation': 'The Beaches', 'headerFullLocation': 'The Beaches, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 4, 'suggestedBounds': {'ne': {'lat': 43.6808574045, 'lng': -79.28682091449052}, 'sw': {'lat': 43.6718573955, 'lng': -79.29924148550948}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bd461bc77b29c74a07d9282', 'name': 'Glen Manor Ravine', 'location': {'address': 'Glen Manor', 'crossStreet': 'Queen St.', 'lat': 43.67682094413784, 'lng': -79.29394208780985, 'labeledLatLngs': [{'label': 'display', 'lat': 43.67682094413784, 'lng': -79.29394208780985}], 'distance': 89, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Can

In [44]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Danforth East,43.685347,-79.338106,Donlands & Sammon,43.685787,-79.339516,Intersection
1,The Danforth East,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park
2,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
3,The Danforth East,43.685347,-79.338106,Donlands & Mortimer,43.687680,-79.340100,Intersection
4,The Danforth East,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store


In [45]:
toronto_venues.to_excel('toronto_venues.xlsx', index=False)

In [3]:
toronto_venues = pd.read_excel('toronto_venues.xlsx', engine='openpyxl')

### Fill Venues-Travel City 2

In [41]:
paris_venues = getNearbyVenues(names=df_paris['neighborhood'],
                                 latitudes=df_paris['latitude'],
                                 longitudes=df_paris['longitude'])

75021 CEDEX 01 - 0
75023 CEDEX 01 - 5
75024 CEDEX 01 - 0
75026 CEDEX 01 - 33
75028 CEDEX 01 - 26
75029 CEDEX 01 - 0
75030 CEDEX 01 - 0
75031 CEDEX 01 - 6
75032 CEDEX 01 - 4
75033 CEDEX 01 - 0
75034 CEDEX 01 - 0
75035 CEDEX 01 - 5
75036 CEDEX 01 - 13
75038 CEDEX 01 - 0
75039 CEDEX 01 - 5
75040 CEDEX 01 - 5
75041 CEDEX 01 - 3
75042 CEDEX 01 - 1
75043 CEDEX 01 - 4
75044 CEDEX 01 - 67
75045 CEDEX 01 - 1
75046 CEDEX 01 - 9
75047 CEDEX 01 - 7
75048 CEDEX 01 - 4
75049 CEDEX 01 - 14
75051 CEDEX 01 - 3
75052 CEDEX 01 - 1
75053 CEDEX 01 - 5
75054 CEDEX 01 - 1
75055 CEDEX 01 - 0
75056 CEDEX 01 - 8
75058 CEDEX 01 - 0
75059 CEDEX 01 - 0
75100 CEDEX 01 - 6
75060 CEDEX 02 - 7
75061 CEDEX 02 - 2
75062 CEDEX 02 - 3
75063 CEDEX 02 - 22
75064 CEDEX 02 - 100
75065 CEDEX 02 - 5
75066 CEDEX 02 - 5
75067 CEDEX 02 - 2
75068 CEDEX 02 - 1
75069 CEDEX 02 - 1
75072 CEDEX 02 - 4
75073 CEDEX 02 - 0
75074 CEDEX 02 - 5
75075 CEDEX 02 - 11
75076 CEDEX 02 - 0
75077 CEDEX 02 - 7
75078 CEDEX 02 - 10
75079 CEDEX 02 - 4
75

75598 CEDEX 12 - 0
75599 CEDEX 12 - 3
75601 CEDEX 12 - 3
75602 CEDEX 12 - 0
75603 CEDEX 12 - 1
75604 CEDEX 12 - 2
75605 CEDEX 12 - 4
75606 CEDEX 12 - 3
75607 CEDEX 12 - 0
75608 CEDEX 12 - 0
75609 CEDEX 12 - 4
75610 CEDEX 12 - 6
75611 CEDEX 12 - 0
75612 CEDEX 12 - 4
75613 CEDEX 12 - 2
75614 CEDEX 12 - 0
75616 CEDEX 12 - 6
75201 CEDEX 13 - 42
75204 CEDEX 13 - 0
75205 CEDEX 13 - 1
75206 CEDEX 13 - 31
75210 CEDEX 13 - 0
75211 CEDEX 13 - 11
75212 CEDEX 13 - 4
75214 CEDEX 13 - 2
75243 CEDEX 13 - 31
75244 CEDEX 13 - 17
75619 CEDEX 13 - 0
75621 CEDEX 13 - 3
75622 CEDEX 13 - 4
75623 CEDEX 13 - 3
75624 CEDEX 13 - 3
75625 CEDEX 13 - 2
75626 CEDEX 13 - 5
75627 CEDEX 13 - 2
75628 CEDEX 13 - 4
75629 CEDEX 13 - 0
75630 CEDEX 13 - 2
75631 CEDEX 13 - 4
75632 CEDEX 13 - 0
75633 CEDEX 13 - 7
75635 CEDEX 13 - 0
75636 CEDEX 13 - 2
75637 CEDEX 13 - 6
75638 CEDEX 13 - 2
75639 CEDEX 13 - 0
75640 CEDEX 13 - 0
75641 CEDEX 13 - 0
75642 CEDEX 13 - 1
75643 CEDEX 13 - 1
75644 CEDEX 13 - 0
75646 CEDEX 13 - 6
75647 C

In [44]:
paris_venues.to_excel('paris_venues.xlsx', index=False)

In [4]:
paris_venues = pd.read_excel('paris_venues.xlsx', engine='openpyxl')

In [65]:
paris_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,75023 CEDEX 01,33.0572,-96.746215,Jimmy's Big Burgers,33.058913,-96.749724,Fast Food Restaurant
1,75023 CEDEX 01,33.0572,-96.746215,Golden Chick,33.058176,-96.750010,American Restaurant
2,75023 CEDEX 01,33.0572,-96.746215,Valero,33.056719,-96.750013,Convenience Store
3,75023 CEDEX 01,33.0572,-96.746215,Goodyear,33.056473,-96.750128,Auto Garage
4,75023 CEDEX 01,33.0572,-96.746215,McDonald's,33.058414,-96.751036,Fast Food Restaurant


### Fill Venues-Resident City

In [45]:
newyork_venues = getNearbyVenues(names=df_newyork['neighborhood'],
                                 latitudes=df_newyork['latitude'],
                                 longitudes=df_newyork['longitude'])

Wakefield - 8
Co-op City - 15
Eastchester - 23
Fieldston - 5
Riverdale - 9
Kingsbridge - 64
Marble Hill - 23
Woodlawn - 27
Norwood - 33
Williamsbridge - 4
Baychester - 23
Pelham Parkway - 26
City Island - 26
Bedford Park - 35
University Heights - 21
Morris Heights - 6
Fordham - 83
East Tremont - 16
West Farms - 25
High  Bridge - 28
Melrose - 25
Mott Haven - 21
Port Morris - 11
Longwood - 6
Hunts Point - 12
Morrisania - 25
Soundview - 17
Clason Point - 11
Throgs Neck - 11
Country Club - 6
Parkchester - 33
Westchester Square - 34
Van Nest - 20
Morris Park - 23
Belmont - 98
Spuyten Duyvil - 11
North Riverdale - 24
Pelham Bay - 37
Schuylerville - 20
Edgewater Park - 24
Castle Hill - 8
Olinville - 10
Pelham Gardens - 21
Concourse - 23
Unionport - 21
Edenwald - 4
Bay Ridge - 81
Bensonhurst - 32
Sunset Park - 38
Greenpoint - 100
Gravesend - 29
Brighton Beach - 43
Sheepshead Bay - 24
Manhattan Terrace - 22
Flatbush - 22
Crown Heights - 21
East Flatbush - 12
Kensington - 34
Windsor Terrace - 26

In [46]:
newyork_venues.to_excel('newyork_venues.xlsx', index=False)

In [49]:
newyork_venues = pd.read_excel('newyork_venues.xlsx', engine='openpyxl')


In [66]:
newyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


## Data Analysis method

As the objective of the analysis is to evaluate the difference in the profiles of venues, the databases were subjected to a transformation so that all the observed commercial categories became columns in the database.

### Building a proportional venue category table

In [50]:
def data_onehot(df_venues, city_name):
    """Create a boolean table using venue category as columns"""
    # one hot encoding
    city_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    city_onehot['city'] = city_name
    
    # move neighborhood column to the first column
    list_columns = list(city_onehot.columns)
    list_columns.remove('city')
    fixed_columns = ['city'] + list_columns
    
    city_onehot = city_onehot[fixed_columns]
    return city_onehot


In [51]:
def data_group_by_city(df_onehot):
    """Calculate the venue proportion from boolean table"""
    city_grouped = df_onehot.groupby('city').mean().reset_index()
    return city_grouped

In [52]:
def get_df_grouped(df_venues, city_name):
    """pipe line process"""
    city_onehot = data_onehot(df_venues, city_name)
    city_grouped = data_group_by_city(city_onehot)
    return city_grouped

In [53]:
def get_transpose_data(city_grouped):
    """Invert table, from column orientation to line orientation"""
    city = city_grouped.loc[0, 'city']
    city_grouped_transpose = city_grouped.transpose().reset_index().loc[1:, :]
    city_grouped_transpose.columns = ['category', city]
    return city_grouped_transpose

#### Analyze-Resident city

In [56]:
newyork_grouped = get_df_grouped(newyork_venues, 'newyork')
print(f'Total Columns: {newyork_grouped.shape[1]}')
newyork_grouped

Total Columns: 384


,city,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pi

In [11]:
newyork_grouped_transpose = get_transpose_data(newyork_grouped)
newyork_grouped_transpose.head()

,category,newyork
1,Accessories Store,0.001156
2,Adult Boutique,0.000165
3,Afghan Restaurant,0.000165
4,African Restaurant,0.000991
5,Airport Terminal,0.000165


#### Analyze-Travel city 1

In [12]:
paris_grouped = get_df_grouped(paris_venues, 'paris')
print(paris_grouped.shape)
paris_grouped

(1, 392)


,city,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Service,Alsatian Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basque Restaurant,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Champagne Bar,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Quad,College Soccer Field,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Pla

In [13]:
paris_grouped_transpose = get_transpose_data(paris_grouped)
paris_grouped_transpose.head()

,category,paris
1,ATM,0.000413
2,Accessories Store,0.000827
3,Adult Boutique,0.000207
4,African Restaurant,0.00124
5,Airport,0.000413


#### Analyze-Travel city 2

In [14]:
toronto_grouped = get_df_grouped(toronto_venues, 'toronto')
print(toronto_grouped.shape)
toronto_grouped

(1, 230)


,city,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,toronto,0.000634,0.000634,0.001267,0.001901,0.001267,0.010139,0.001901,0.003169,0.007605,0.001267,0.001267,0.005703,0.000634,0.001267,0.000634,0.002535,0.020913,0.007605,0.014575,0.001267,0.002535,0.000634,0.000634,0.010773,0.002535,0.002535,0.003169,0.000634,0.011407,0.000634,0.001267,0.010773,0.007605,0.00507,0.000634,0.006337,0.005703,0.000634,0.000634,0.000634,0.054499,0.000634,0.000634,0.002535,0.003802,0.00507,0.000634,0.000634,0.000634,0.012674,0.009506,0.091255,0.000634,0.000634,0.000634,0.001267,0.003802,0.001901,0.005703,0.002535,0.006971,0.000634,0.004436,0.001267,0.001267,0.001267,0.009506,0.007605,0.006337,0.008872,0.001901,0.001267,0.001267,0.000634,0.001267,0.000634,0.001901,0.003169,0.000634,

In [15]:
toronto_grouped_transpose = get_transpose_data(toronto_grouped)
toronto_grouped_transpose.head()

,category,toronto
1,Airport,0.000634
2,Airport Food Court,0.000634
3,Airport Lounge,0.001267
4,Airport Service,0.001901
5,Airport Terminal,0.001267


### Unify tables

In [17]:
def get_serie_category_unify(list_dfs_cities):
    
    df_cat = pd.DataFrame()
    df_cat['category'] = ''
    list_series = []
    for df in list_dfs_cities:
        list_series.append(df['category'])
    df_cat['category'] = pd.concat(list_series)
    serie_category_unprocess = df_cat['category']
    serie_category = serie_category_unprocess.unique()
    return serie_category

list_dfs_cities = [newyork_grouped_transpose, paris_grouped_transpose, toronto_grouped_transpose]
serie_category = get_serie_category_unify(list_dfs_cities)
print(serie_category.shape)
serie_category[:5]

(492,)


array(['Accessories Store', 'Adult Boutique', 'Afghan Restaurant',
       'African Restaurant', 'Airport Terminal'], dtype=object)

In [18]:
def get_cities_freq_matrix(list_dfs_cities, serie_category):
    
    df_merge_unprocess = pd.DataFrame()
    df_merge_unprocess['category'] = serie_category
    
    for df in list_dfs_cities:
        df_merge_unprocess = df_merge_unprocess.join(df.set_index('category'), on='category', how='left')
    
    # replace N/A for 0
    df_merge = df_merge_unprocess.fillna(0).reset_index(drop=True)
    return df_merge

cities_freq_matrix = get_cities_freq_matrix(list_dfs_cities, serie_category)
print(cities_freq_matrix.shape)
cities_freq_matrix

(492, 4)


,category,newyork,paris,toronto
0,Accessories Store,0.001156,0.000827,0.000000
1,Adult Boutique,0.000165,0.000207,0.000000
2,Afghan Restaurant,0.000165,0.000000,0.000000
3,African Restaurant,0.000991,0.001240,0.000000
4,Airport Terminal,0.000165,0.000000,0.001267
5,American Restaurant,0.018164,0.009093,0.010139
6,Antique Shop,0.000661,0.000827,0.001901
7,Arcade,0.000165,0.000413,0.000000
8,Arepa Restaurant,0.000826,0.000207,0.000000
9,Argentinian Restaurant,0.001156,0.000413,0.000000


### Distinction Function

To calculate the degree of distinction of the venues profiles of the city, the function was taken:

**Variables:**\
Being X, the proportion of existence of a commercial category in the residential city;\
Y, the proportion of existence of a commercial category in a city for travel;\
Z, the function of degree of distinction of 1 category between 2 cities;\
W,  the function of the degree of distinction of the commercial categories of 2 cities.

**Function:**
x: Proportion of venue category in resident city\
y: Proportion of venue category in destination city\
z: `1-[min(x, y) / max(x, y)`\
w: `mean(∑z)`\
Note: if min(x, y) == 0, then z == 1

Special cases:
- In cases where the venue category exists in the resident city and does not exist in the destination city, the distinction function would not be awarded, the category being discarded from the analysis between the two cities.
- In cases where the commercial category does not exist in the resident city, but exist in the destination city, variable Z would be considered as 1. Maximum degree of distinction.

### Applying Distinction function

In [63]:
def get_distinction_category(df, resident_city, travel_cities):
    
    df_distinction_category= df[['category']]
    for travel_city in travel_cities:
        
        # create empty column in df_distinction to travel city
        df_distinction_category[travel_city] = 0
        
        df_select = df.loc[:, [resident_city, travel_city]]
        for index, item in df_select.iterrows():
            value1 = df_select.loc[index, resident_city]
            value2 = df_select.loc[index, travel_city]
            if round(value1, 6) == round(value2, 6) and round(value1, 6) == 0:
                distinction = '-'
            else:
                if min(value1, value2) == 0:
                    if value1 == 0:
                        distinction = 1
                    else:
                        distinction = '-'
                else:
                    distinction = 1-(min(value1, value2) / max(value1, value2))
            # fill df_distinction_category for travel city
            df_distinction_category.loc[index, travel_city] = distinction
    
    return df_distinction_category

df_distinction_category = get_distinction_category(df=cities_freq_matrix, resident_city='newyork', 
                                                   travel_cities=['paris', 'toronto'])
print(f'Categories count: {df_distinction_category.shape[0]}')
print('Similarity rate from Resident City')
df_distinction_category

Categories count: 492
Similarity rate from Resident City


,category,paris,toronto
0,Accessories Store,0.284858,-
1,Adult Boutique,0.200958,-
2,Afghan Restaurant,-,-
3,African Restaurant,0.200958,-
4,Airport Terminal,-,0.869716
5,American Restaurant,0.499401,0.441779
6,Antique Shop,0.200958,0.652576
7,Arcade,0.600479,-
8,Arepa Restaurant,0.7497,-
9,Argentinian Restaurant,0.642429,-


In [44]:
def get_df_distinction(df_distinction_category, resident_city, travel_cities):
    
    df = df_distinction_category.copy()
    l=[]
    d={}

    d['city'] = resident_city
    for travel in travel_cities:
        mask = df[travel].isin(['-'])
        serie_filter_na =  df.loc[~mask, travel]
        serie_travel = serie_filter_na
        d['distinction_' + travel] = serie_travel.mean()
    l.append(d)
    df_distinction = pd.DataFrame(l)
    return df_distinction



,city,distinction_paris,distinction_toronto
0,newyork,0.610201,0.54663


## Results

It was observed that from the resident city of New York, the city of Paris presents a greater degree of distinction (61%) in commercial establishments than the city of Toronto (54.6%).

Interestingly, it is worth mentioning that one of the factors of strong influence for this result, was the consideration of the special cases highlighted in the model. If the absence of any venue category in one of the cities represented a degree of maximum distinction for that category, without any caveat about the source of this non-existence, the city of Toronto would be 74.8% more distinct, compared to 67.8% to Paris.


In [67]:
df_distinction = get_df_distinction(df_distinction_category, resident_city='newyork', travel_cities=['paris', 'toronto'])
df_distinction

,city,distinction_paris,distinction_toronto
0,newyork,0.610201,0.54663


Despite the diversity of destination cities under analysis for decision making having been only two, the model is not restricted by the number of cities and can be used to guide decisions in scenarios with greater options.